In [1]:
pip install mysql-connector-python azure-storage-blob pyspark


   ---------------------------------------- 0.0/396.4 kB ? eta -:--:--
   - -------------------------------------- 10.2/396.4 kB ? eta -:--:--
   --------- ------------------------------ 92.2/396.4 kB 1.3 MB/s eta 0:00:01
   ----------------- ---------------------- 174.1/396.4 kB 1.7 MB/s eta 0:00:01
   --------------------------- ------------ 276.5/396.4 kB 1.7 MB/s eta 0:00:01
   ------------------------------------- -- 368.6/396.4 kB 1.8 MB/s eta 0:00:01
   ---------------------------------------- 396.4/396.4 kB 1.7 MB/s eta 0:00:00
   ---------------------------------------- 0.0/194.3 kB ? eta -:--:--
   ------ --------------------------------- 30.7/194.3 kB 1.3 MB/s eta 0:00:01
   -------- ------------------------------ 41.0/194.3 kB 487.6 kB/s eta 0:00:01
   -------------- ------------------------ 71.7/194.3 kB 558.5 kB/s eta 0:00:01
   ------------------ -------------------- 92.2/194.3 kB 525.1 kB/s eta 0:00:01
   ------------------ -------------------- 92.2/194.3 kB 525.1 kB/s 

In [5]:
import mysql.connector
from mysql.connector import Error
import pandas as pd
from azure.storage.blob import BlobServiceClient, BlobClient, ContainerClient
import os

# MySQL connection details
mysql_config = {
    'user': 'root',
    'password': 'Harry@1995',
    'host': 'localhost',
    'database': 'cap_stone_new'
}

# ADLS connection details
connection_string = "DefaultEndpointsProtocol=https;AccountName=sqlexternalto;AccountKey=H90pP7GomvGQbr1Z36IELtsDgYnYK3jn2kpbu4hmaD5GMU6pR/6h4z5D86X0dMrZDMot7rct0/AD+AStVFOTUg==;EndpointSuffix=core.windows.net"
container_name = "rawdata"
blob_service_client = BlobServiceClient.from_connection_string(connection_string)
container_client = blob_service_client.get_container_client(container_name)

# Directory where the CSV files will be saved
local_directory = r"C:\Users\saraswathira\OneDrive - Maveric Systems Limited\Desktop\Cap_Stone_Databricks\output"

# Ensure the local directory exists
if not os.path.exists(local_directory):
    os.makedirs(local_directory)

# List of MySQL tables and corresponding CSV filenames and ADLS directories
tables_and_directories = [
    ("branches", "branches.csv", "branches_directory"),
    ("customers", "customers.csv", "customers_directory"),
    ("transactions", "transactions.csv", "transactions_directory")
]

def fetch_data_from_mysql(query):
    try:
        connection = mysql.connector.connect(**mysql_config)
        if connection.is_connected():
            cursor = connection.cursor()
            cursor.execute(query)
            columns = [desc[0] for desc in cursor.description]
            data = cursor.fetchall()
            return pd.DataFrame(data, columns=columns)
    except Error as e:
        print(f"Error connecting to MySQL: {e}")
    finally:
        if connection.is_connected():
            cursor.close()
            connection.close()

def upload_file_to_directory(file_path, directory_name):
    file_name = os.path.basename(file_path)
    blob_name = f"{directory_name}/{file_name}"
    blob_client = container_client.get_blob_client(blob_name)

    with open(file_path, "rb") as data:
        blob_client.upload_blob(data)
    print(f"Uploaded {file_name} to {blob_name}")

# Fetch data from MySQL, save to CSV, and upload to ADLS
for table, csv_file, directory in tables_and_directories:
    query = f"SELECT * FROM {table}"
    df = fetch_data_from_mysql(query)
    
    # Save the DataFrame to a CSV file
    csv_path = os.path.join(local_directory, csv_file)
    df.to_csv(csv_path, index=False)
    print(f"Saved {table} data to {csv_path}")

    # Ensure the directory exists in the container
    directory_client = container_client.get_blob_client(f"{directory}/.keep")
    try:
        directory_client.upload_blob(b"", overwrite=True)  # Create an empty blob to create the directory
    except Exception as e:
        print(f"Error creating directory {directory}: {e}")

    # Upload the CSV file to the respective directory in ADLS
    try:
        upload_file_to_directory(csv_path, directory)
    except Exception as e:
        print(f"Error uploading file {csv_path} to directory {directory}: {e}")


Saved branches data to C:\Users\saraswathira\OneDrive - Maveric Systems Limited\Desktop\Cap_Stone_Databricks\output\branches.csv
Error uploading file C:\Users\saraswathira\OneDrive - Maveric Systems Limited\Desktop\Cap_Stone_Databricks\output\branches.csv to directory branches_directory: The specified blob already exists.
RequestId:bcad5203-d01e-0031-4580-decb2e000000
Time:2024-07-25T10:50:20.5376792Z
ErrorCode:BlobAlreadyExists
Content: <?xml version="1.0" encoding="utf-8"?><Error><Code>BlobAlreadyExists</Code><Message>The specified blob already exists.
RequestId:bcad5203-d01e-0031-4580-decb2e000000
Time:2024-07-25T10:50:20.5376792Z</Message></Error>
Saved customers data to C:\Users\saraswathira\OneDrive - Maveric Systems Limited\Desktop\Cap_Stone_Databricks\output\customers.csv
Error uploading file C:\Users\saraswathira\OneDrive - Maveric Systems Limited\Desktop\Cap_Stone_Databricks\output\customers.csv to directory customers_directory: The specified blob already exists.
RequestId:bc

###Final

In [36]:
import mysql.connector
from mysql.connector import Error
import pandas as pd
from azure.storage.blob import BlobServiceClient, BlobClient, ContainerClient
import os

# MySQL connection details
mysql_config = {
    'user': 'root',
    'password': 'Harry@1995',
    'host': 'localhost',
    'database': 'cap_stone_new'
}

# ADLS connection details
connection_string = "DefaultEndpointsProtocol=https;AccountName=sqlexternalto;AccountKey=H90pP7GomvGQbr1Z36IELtsDgYnYK3jn2kpbu4hmaD5GMU6pR/6h4z5D86X0dMrZDMot7rct0/AD+AStVFOTUg==;EndpointSuffix=core.windows.net"
container_name = "rawdata"
blob_service_client = BlobServiceClient.from_connection_string(connection_string)
container_client = blob_service_client.get_container_client(container_name)

# Directory where the CSV files will be saved
local_directory = r"C:\Users\saraswathira\OneDrive - Maveric Systems Limited\Desktop\Cap_Stone_Databricks\output"

# Ensure the local directory exists
if not os.path.exists(local_directory):
    os.makedirs(local_directory)

# List of MySQL tables and corresponding CSV filenames and ADLS directories
tables_and_directories = [
    ("branches", "branches.csv", "branches_directory"),
    ("customers", "customers.csv", "customers_directory"),
    ("transactions", "transactions.csv", "transactions_directory")
]

def fetch_data_from_mysql(query):
    try:
        connection = mysql.connector.connect(**mysql_config)
        if connection.is_connected():
            cursor = connection.cursor()
            cursor.execute(query)
            columns = [desc[0] for desc in cursor.description]
            data = cursor.fetchall()
            return pd.DataFrame(data, columns=columns)
    except Error as e:
        print(f"Error connecting to MySQL: {e}")
    finally:
        if connection.is_connected():
            cursor.close()
            connection.close()

def upload_file_to_directory(file_path, directory_name):
    file_name = os.path.basename(file_path)
    blob_name = f"{directory_name}/{file_name}"
    blob_client = container_client.get_blob_client(blob_name)

    with open(file_path, "rb") as data:
        blob_client.upload_blob(data, overwrite=True)  # Overwrite existing blob if it exists
    print(f"Uploaded {file_name} to {blob_name}")

# Fetch data from MySQL, save to CSV, and upload to ADLS
for table, csv_file, directory in tables_and_directories:
    query = f"SELECT * FROM {table}"
    df = fetch_data_from_mysql(query)
    
    # Save the DataFrame to a CSV file
    csv_path = os.path.join(local_directory, csv_file)
    df.to_csv(csv_path, index=False)
    print(f"Saved {table} data to {csv_path}")

    # Ensure the directory exists in the container
    directory_client = container_client.get_blob_client(f"{directory}/.keep")
    try:
        directory_client.upload_blob(b"", overwrite=True)  # Create an empty blob to ensure the directory exists
    except Exception as e:
        print(f"Error creating directory {directory}: {e}")

    # Upload the CSV file to the respective directory in ADLS
    try:
        upload_file_to_directory(csv_path, directory)
    except Exception as e:
        print(f"Error uploading file {csv_path} to directory {directory}: {e}")


Saved branches data to C:\Users\saraswathira\OneDrive - Maveric Systems Limited\Desktop\Cap_Stone_Databricks\output\branches.csv
Uploaded branches.csv to branches_directory/branches.csv
Saved customers data to C:\Users\saraswathira\OneDrive - Maveric Systems Limited\Desktop\Cap_Stone_Databricks\output\customers.csv
Uploaded customers.csv to customers_directory/customers.csv
Saved transactions data to C:\Users\saraswathira\OneDrive - Maveric Systems Limited\Desktop\Cap_Stone_Databricks\output\transactions.csv
Uploaded transactions.csv to transactions_directory/transactions.csv


In [35]:
import mysql.connector
from mysql.connector import Error
import pandas as pd
from azure.storage.blob import BlobServiceClient, BlobClient, ContainerClient
import os
from datetime import datetime
import json

# MySQL connection details
mysql_config = {
    'user': 'root',
    'password': 'Harry@1995',
    'host': 'localhost',
    'database': 'cap_stone_new'
}

# ADLS connection details
connection_string = "DefaultEndpointsProtocol=https;AccountName=sqlexternalto;AccountKey=H90pP7GomvGQbr1Z36IELtsDgYnYK3jn2kpbu4hmaD5GMU6pR/6h4z5D86X0dMrZDMot7rct0/AD+AStVFOTUg==;EndpointSuffix=core.windows.net"
container_name = "rawdata"
blob_service_client = BlobServiceClient.from_connection_string(connection_string)
container_client = blob_service_client.get_container_client(container_name)

# Directory where the CSV files will be saved
local_directory = r"C:\Users\saraswathira\OneDrive - Maveric Systems Limited\Desktop\Cap_Stone_Databricks\output"

# Ensure the local directory exists
if not os.path.exists(local_directory):
    os.makedirs(local_directory)

# List of MySQL tables and corresponding CSV filenames and ADLS directories
tables_and_directories = [
    ("branches", "branches.csv", "branches_directory"),
    ("customers", "customers.csv", "customers_directory"),
    ("transactions", "transactions.csv", "transactions_directory")
]

# File to store the last processed timestamps
last_processed_file = os.path.join(local_directory, "last_processed.json")

# Read the last processed timestamps
if os.path.exists(last_processed_file):
    with open(last_processed_file, "r") as f:
        last_processed = json.load(f)
else:
    last_processed = {}

def fetch_data_from_mysql(query):
    try:
        connection = mysql.connector.connect(**mysql_config)
        if connection.is_connected():
            cursor = connection.cursor()
            cursor.execute(query)
            columns = [desc[0] for desc in cursor.description]
            data = cursor.fetchall()
            return pd.DataFrame(data, columns=columns)
    except Error as e:
        print(f"Error connecting to MySQL: {e}")
    finally:
        if connection.is_connected():
            cursor.close()
            connection.close()

def upload_file_to_directory(file_path, directory_name):
    file_name = os.path.basename(file_path)
    blob_name = f"{directory_name}/{file_name}"
    blob_client = container_client.get_blob_client(blob_name)

    with open(file_path, "rb") as data:
        blob_client.upload_blob(data, overwrite=True)  # Overwrite existing blob if it exists
    print(f"Uploaded {file_name} to {blob_name}")

def check_if_file_exists(directory_name, file_name):
    blob_name = f"{directory_name}/{file_name}"
    blob_client = container_client.get_blob_client(blob_name)
    return blob_client.exists()

# Fetch data from MySQL, save to CSV, and upload to ADLS
for table, csv_file, directory in tables_and_directories:
    if table == "branches":
        # Check if the branches file already exists in ADLS
        if check_if_file_exists(directory, csv_file):
            print(f"{csv_file} already exists in {directory}. Skipping.")
            continue

    last_processed_value = last_processed.get(table, None)
    if last_processed_value and table != "branches":
        query = f"SELECT * FROM {table} WHERE last_update > '{last_processed_value}'"
    else:
        query = f"SELECT * FROM {table}"

    print(f"Fetching data for table: {table} with query: {query}")
    df = fetch_data_from_mysql(query)
    print(f"Fetched {len(df)} rows for table: {table}")

    if df.empty:
        print(f"No new data found for table: {table}")
        continue

    # Generate a unique filename with a timestamp (for customers and transactions)
    if table != "branches":
        timestamp = datetime.now().strftime("%Y%m%d%H%M%S")
        unique_csv_file = f"{os.path.splitext(csv_file)[0]}_{timestamp}.csv"
    else:
        unique_csv_file = csv_file
    
    # Save the DataFrame to a CSV file
    csv_path = os.path.join(local_directory, unique_csv_file)
    df.to_csv(csv_path, index=False)
    print(f"Saved {table} data to {csv_path}")

    # Ensure the directory exists in the container
    directory_client = container_client.get_blob_client(f"{directory}/.keep")
    try:
        directory_client.upload_blob(b"", overwrite=True)  # Create an empty blob to ensure the directory exists
    except Exception as e:
        print(f"Error creating directory {directory}: {e}")

    # Upload the CSV file to the respective directory in ADLS
    try:
        upload_file_to_directory(csv_path, directory)
    except Exception as e:
        print(f"Error uploading file {csv_path} to directory {directory}: {e}")

    # Update the last processed timestamp
    if table != "branches":
        if 'last_update' in df.columns:
            last_processed[table] = df['last_update'].max().strftime('%Y-%m-%d %H:%M:%S')
        elif 'id' in df.columns:
            last_processed[table] = str(df['id'].max())

# Save the last processed timestamps
with open(last_processed_file, "w") as f:
    json.dump(last_processed, f)


Fetching data for table: branches with query: SELECT * FROM branches
Fetched 10 rows for table: branches
Saved branches data to C:\Users\saraswathira\OneDrive - Maveric Systems Limited\Desktop\Cap_Stone_Databricks\output\branches.csv
Uploaded branches.csv to branches_directory/branches.csv
Fetching data for table: customers with query: SELECT * FROM customers WHERE last_update > '2023-12-30 00:00:00'
Fetched 0 rows for table: customers
No new data found for table: customers
Fetching data for table: transactions with query: SELECT * FROM transactions
Fetched 300 rows for table: transactions
Saved transactions data to C:\Users\saraswathira\OneDrive - Maveric Systems Limited\Desktop\Cap_Stone_Databricks\output\transactions_20240731184030.csv
Uploaded transactions_20240731184030.csv to transactions_directory/transactions_20240731184030.csv


In [44]:
import mysql.connector
from mysql.connector import Error
import pandas as pd
from azure.storage.blob import BlobServiceClient
import os
from datetime import datetime
import json

# MySQL connection details
mysql_config = {
    'user': 'root',
    'password': 'Harry@1995',
    'host': 'localhost',
    'database': 'cap_stone_new'
}

# ADLS connection details
connection_string = "DefaultEndpointsProtocol=https;AccountName=sqlexternalto;AccountKey=H90pP7GomvGQbr1Z36IELtsDgYnYK3jn2kpbu4hmaD5GMU6pR/6h4z5D86X0dMrZDMot7rct0/AD+AStVFOTUg==;EndpointSuffix=core.windows.net"
container_name = "rawdata"

# Directory where the CSV files will be saved
local_directory = r"C:\Users\saraswathira\OneDrive - Maveric Systems Limited\Desktop\Cap_Stone_Databricks\output"

# List of MySQL tables and corresponding CSV filenames and ADLS directories
tables_and_directories = [
    ("branches", "branches.csv", "branches_directory", "branch_id"),
    ("customers", "customers.csv", "customers_directory", "customer_id"),
    ("transactions", "transactions.csv", "transactions_directory", "transaction_id")
]

# Directory to store checkpoint files
checkpoint_directory = os.path.join(local_directory, "checkpoints")
if not os.path.exists(checkpoint_directory):
    os.makedirs(checkpoint_directory)

# Initialize ADLS Blob Service Client
blob_service_client = BlobServiceClient.from_connection_string(connection_string)

def fetch_data_from_mysql(query):
    """Fetch data from MySQL using the given query."""
    try:
        connection = mysql.connector.connect(**mysql_config)
        if connection.is_connected():
            cursor = connection.cursor()
            cursor.execute(query)
            columns = [desc[0] for desc in cursor.description]
            data = cursor.fetchall()
            return pd.DataFrame(data, columns=columns)
    except Error as e:
        print(f"Error connecting to MySQL: {e}")
    finally:
        if connection.is_connected():
            cursor.close()
            connection.close()

def upload_file_to_directory(file_path, directory_name):
    """Upload a file to the specified directory in ADLS."""
    file_name = os.path.basename(file_path)
    blob_name = f"{directory_name}/{file_name}"
    blob_client = blob_service_client.get_blob_client(container=container_name, blob=blob_name)

    with open(file_path, "rb") as data:
        blob_client.upload_blob(data, overwrite=True)
    print(f"Uploaded {file_name} to {blob_name}")

def check_if_file_exists(directory_name, file_name):
    """Check if a file exists in the specified directory in ADLS."""
    blob_name = f"{directory_name}/{file_name}"
    blob_client = blob_service_client.get_blob_client(container=container_name, blob=blob_name)
    return blob_client.exists()

def create_local_directory():
    """Create the local directory if it does not exist."""
    if not os.path.exists(local_directory):
        os.makedirs(local_directory)

def read_checkpoint(table):
    """Read checkpoint data from file."""
    checkpoint_file = os.path.join(checkpoint_directory, f"{table}_checkpoint.json")
    if os.path.exists(checkpoint_file):
        with open(checkpoint_file, "r") as f:
            return json.load(f)
    return {"last_id": ""}  # Initialize with an empty string

def save_checkpoint(table, checkpoint_data):
    """Save checkpoint data to file."""
    checkpoint_file = os.path.join(checkpoint_directory, f"{table}_checkpoint.json")
    with open(checkpoint_file, "w") as f:
        json.dump(checkpoint_data, f)

def generate_unique_filename(file_name, table):
    """Generate a unique filename based on the table and current timestamp."""
    if table != "branches":
        timestamp = datetime.now().strftime("%Y%m%d%H%M%S")
        return f"{os.path.splitext(file_name)[0]}_{timestamp}.csv"
    return file_name

def ensure_directory_exists(directory_name):
    """Ensure the directory exists in ADLS by creating a dummy blob."""
    directory_client = blob_service_client.get_blob_client(container=container_name, blob=f"{directory_name}/.keep")
    try:
        directory_client.upload_blob(b"", overwrite=True)
    except Exception as e:
        print(f"Error creating directory {directory_name}: {e}")

def process_table(table, csv_file, directory, primary_key):
    """Process a table by fetching data, saving to CSV, and uploading to ADLS."""
    checkpoint_data = read_checkpoint(table)
    last_processed_value = checkpoint_data.get('last_id', "")

    query = f"SELECT * FROM {table} WHERE {primary_key} > '{last_processed_value}'"
    
    print(f"Fetching data for table: {table} with query: {query}")
    df = fetch_data_from_mysql(query)
    print(f"Fetched {len(df)} rows for table: {table}")

    if df.empty:
        print(f"No new data found for table: {table}")
        return

    unique_csv_file = generate_unique_filename(csv_file, table)
    csv_path = os.path.join(local_directory, unique_csv_file)
    df.to_csv(csv_path, index=False)
    print(f"Saved {table} data to {csv_path}")

    ensure_directory_exists(directory)
    
    if not check_if_file_exists(directory, unique_csv_file):
        try:
            upload_file_to_directory(csv_path, directory)
        except Exception as e:
            print(f"Error uploading file {csv_path} to directory {directory}: {e}")

    if primary_key in df.columns:
        checkpoint_data['last_id'] = df[primary_key].max()
        save_checkpoint(table, checkpoint_data)

# Main script execution
create_local_directory()

for table, csv_file, directory, primary_key in tables_and_directories:
    process_table(table, csv_file, directory, primary_key)


Fetching data for table: branches with query: SELECT * FROM branches WHERE branch_id > 'B0009'
Fetched 0 rows for table: branches
No new data found for table: branches
Fetching data for table: customers with query: SELECT * FROM customers WHERE customer_id > 'C1199'
Fetched 0 rows for table: customers
No new data found for table: customers
Fetching data for table: transactions with query: SELECT * FROM transactions WHERE transaction_id > 'T5598'
Fetched 0 rows for table: transactions
No new data found for table: transactions
